In [8]:
import numpy as np

# 设置随机种子，保证每次运行结果相同
np.random.seed(42)

# 序列长度
T = 10  
# 输入维度
input_size = 1  
# 隐藏层维度
hidden_size = 8  
# 输出维度
output_size = 1  

# 生成随机输入序列 x: shape = (T, input_size)
x_data = np.random.randn(T, input_size)
# 生成目标输出序列 y: 这里我们人为设定 y = 2 * x
y_data = 2 * x_data

print("输入序列 x_data:\n", x_data)
print("目标输出 y_data:\n", y_data)


输入序列 x_data:
 [[ 0.49671415]
 [-0.1382643 ]
 [ 0.64768854]
 [ 1.52302986]
 [-0.23415337]
 [-0.23413696]
 [ 1.57921282]
 [ 0.76743473]
 [-0.46947439]
 [ 0.54256004]]
目标输出 y_data:
 [[ 0.99342831]
 [-0.2765286 ]
 [ 1.29537708]
 [ 3.04605971]
 [-0.46830675]
 [-0.46827391]
 [ 3.15842563]
 [ 1.53486946]
 [-0.93894877]
 [ 1.08512009]]


In [9]:
class RNN:
    def __init__(self, input_size, hidden_size, output_size, lr=0.01):
        """
        初始化权重和偏置
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = lr  # 学习率

        # 权重初始化（这里使用较小的随机值）
        self.W_xh = np.random.randn(input_size, hidden_size) * 0.01
        self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.W_hy = np.random.randn(hidden_size, output_size) * 0.01
        
        # 偏置初始化为 0
        self.b_h = np.zeros((hidden_size,))
        self.b_y = np.zeros((output_size,))
        
    def forward(self, x_seq):
        """
        前向传播（对一个序列 x_seq），返回：
        1. h_seq: 每个时刻的隐藏状态列表
        2. y_seq: 每个时刻的输出列表
        """
        # 记录每个时刻的隐藏状态和输出
        h_seq = []
        y_seq = []
        
        # 初始化隐藏状态 h(0) = 0
        h_t = np.zeros((self.hidden_size,))
        
        for x_t in x_seq:
            # x_t shape = (input_size,)，这里把它当成 1D 向量
            
            # 计算新的隐藏状态
            h_t = np.tanh(
                x_t @ self.W_xh + 
                h_t @ self.W_hh + 
                self.b_h
            )
            
            # 计算输出
            y_t = h_t @ self.W_hy + self.b_y
            
            # 记录下来
            h_seq.append(h_t)
            y_seq.append(y_t)
        
        # 转成 numpy 数组
        h_seq = np.stack(h_seq, axis=0)  # shape = (T, hidden_size)
        y_seq = np.stack(y_seq, axis=0)  # shape = (T, output_size)
        return h_seq, y_seq
    
    def compute_loss(self, y_pred, y_true):
        """
        计算均方误差损失 (MSE)
        y_pred, y_true 形状相同: (T, output_size)
        """
        return 0.5 * np.sum((y_pred - y_true)**2)
    
    def backward(self, x_seq, h_seq, y_seq, y_true):
        """
        反向传播，通过 BPTT 对参数求梯度
        x_seq, h_seq, y_seq, y_true 的形状/列表对应 forward 函数的输出
        """
        T = len(x_seq)
        
        # 初始化梯度
        dW_xh = np.zeros_like(self.W_xh)
        dW_hh = np.zeros_like(self.W_hh)
        dW_hy = np.zeros_like(self.W_hy)
        db_h  = np.zeros_like(self.b_h)
        db_y  = np.zeros_like(self.b_y)
        
        # 对隐藏状态的梯度初始化为 0
        dh_next = np.zeros((self.hidden_size,))
        
        # 从最后一个时刻往前计算梯度（BPTT）
        for t in reversed(range(T)):
            # 1. 计算输出层梯度
            dy = (y_seq[t] - y_true[t])  # shape = (output_size,)
            dW_hy += np.outer(h_seq[t], dy)
            db_y  += dy
            
            # 2. 计算对隐藏状态的梯度（从输出层往回）
            dh = dy @ self.W_hy.T + dh_next  # 对隐藏状态的总梯度
            # 因为 h_t = tanh(...)，所以对 tanh 的梯度：dtanh(z) = (1 - tanh^2(z))
            dtanh = (1 - h_seq[t]**2) * dh
            
            # 3. 计算对各项权重和偏置的梯度
            # x_seq[t] shape = (input_size,)
            dW_xh += np.outer(x_seq[t], dtanh)
            
            # 如果 t == 0，h_seq[t-1] 不存在，视为 0 向量
            h_prev = np.zeros_like(h_seq[t]) if t == 0 else h_seq[t-1]
            dW_hh += np.outer(h_prev, dtanh)
            db_h  += dtanh
            
            # 4. 计算对前一个隐藏状态的梯度，用于继续 BPTT
            dh_next = dtanh @ self.W_hh.T
        
        # 5. 梯度下降更新参数
        self.W_xh -= self.lr * dW_xh
        self.W_hh -= self.lr * dW_hh
        self.W_hy -= self.lr * dW_hy
        self.b_h  -= self.lr * db_h
        self.b_y  -= self.lr * db_y

    def train_step(self, x_seq, y_true):
        """
        完整的单次前向传播 + 反向传播 + 参数更新
        """
        # 前向传播
        h_seq, y_seq = self.forward(x_seq)
        # 计算损失
        loss = self.compute_loss(y_seq, y_true)
        # 反向传播
        self.backward(x_seq, h_seq, y_seq, y_true)
        
        return loss


In [10]:
# 实例化 RNN
rnn = RNN(input_size, hidden_size, output_size, lr=0.01)

# 训练轮数
num_epochs = 200

for epoch in range(num_epochs):
    loss = rnn.train_step(x_data, y_data)
    if (epoch+1) % 20 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss = {loss:.4f}")

# 查看最终预测
_, y_pred = rnn.forward(x_data)
print("预测值 y_pred:\n", y_pred)
print("真实值 y_data:\n", y_data)


Epoch 20/200, Loss = 9.2975
Epoch 40/200, Loss = 5.6213
Epoch 60/200, Loss = 0.3721
Epoch 80/200, Loss = 0.0429
Epoch 100/200, Loss = 0.0329
Epoch 120/200, Loss = 0.0289
Epoch 140/200, Loss = 0.0255
Epoch 160/200, Loss = 0.0227
Epoch 180/200, Loss = 0.0204
Epoch 200/200, Loss = 0.0184
预测值 y_pred:
 [[ 1.04348767]
 [-0.32104181]
 [ 1.36640077]
 [ 2.98100245]
 [-0.51198748]
 [-0.5150128 ]
 [ 3.06108598]
 [ 1.61297415]
 [-0.94754366]
 [ 1.14039697]]
真实值 y_data:
 [[ 0.99342831]
 [-0.2765286 ]
 [ 1.29537708]
 [ 3.04605971]
 [-0.46830675]
 [-0.46827391]
 [ 3.15842563]
 [ 1.53486946]
 [-0.93894877]
 [ 1.08512009]]
